In [ ]:
#Importing the Pandas, Numpy, Matplotlib and Seaborn Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px

In [ ]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', None)
sns.set_style('darkgrid')

In [ ]:
#Loading the dataset
df = pd.read_csv('loan.csv')

# Understanding the Data

In [ ]:
# Describing the data

df.describe()

## Step 1: Cleaning

### Choosing columns
There are columns with `Nan` values.

In [ ]:
df.isna().sum()

As seen above, there are columns which do not contain any values. Some of the columns contain not but few values.

In [ ]:
(df.isna().sum() >= 0.5 * df.shape[0]).sum()

57 columns have more than 50% vlues as null. I am choosing not to use them and dropping them from analysis.

In [ ]:
column_names = df.columns[df.isna().sum() <= 0.50 * df.shape[0]].tolist()
data_df = df[column_names]
del df
data_df.sample(3)

Another thing to check in the remaining column is the numbeor of unique values found in the each column

In [ ]:
data_df.nunique()

There are columns which contain only 1 value for all the rows. We are choosing not to use them either as they are not adding any information wrt to our target variable

In [ ]:
(data_df.nunique() <= 1).sum()

9 columns will additionally be removed from our dataset

In [ ]:
column_names = data_df.columns[data_df.nunique() > 1]
data_df = data_df[column_names]
data_df.sample(3)

In [ ]:
data_df.info()

In [ ]:
def get_earliest_yr(x):
    x = int(x.split("-")[1])
    if 0 <= x <= 11:
        return str(2000 + x)
    return str(1900 + x)

In [ ]:
data_df['term'] = data_df.term.apply(lambda x: int(x.strip().split(' ')[0]))
data_df['int_rate'] = data_df.int_rate.apply(lambda x: float(x.strip().strip('%')))

In [ ]:
data_df['issue_d_month'] = data_df.issue_d.apply(lambda x: x.split('-')[0])
data_df['issue_d_year'] = data_df.issue_d.apply(lambda x: "20" + x.split('-')[1])
data_df = data_df.drop(['issue_d'], axis=1)

In [ ]:
data_df['earliest_cr_line_month'] = data_df.earliest_cr_line.apply(lambda x: x.split('-')[0])
data_df['earliest_cr_line_year'] = data_df.earliest_cr_line.apply(get_earliest_yr)
data_df = data_df.drop(['earliest_cr_line'], axis=1)

In [ ]:
data_df.info()

## Step 2 Univariate Analysis

### Annual Income

In [ ]:
data_df.annual_inc.describe()

In [ ]:
plt.figure(figsize=(3,8))
sns.boxplot(data_df.annual_inc)
plt.title('Annual income')
plt.show()

Here we encounter some outliers. It is evident from the 5 point summary that 75% of the annual incomes are below USD 100K, but the highest income is USD 6M. Analyzing this columns while considering all the values would not yield correct results and therefore we choose drop rows where the annual income is greater than USD 150K.

In [ ]:
data_df = data_df[(data_df.annual_inc <= 150_000)]
data_df.annual_inc.describe()

In [ ]:
plt.figure(figsize=(3, 8))
sns.boxplot(data_df.annual_inc)
plt.title('Annual Income')
plt.xticks(ticks=[])
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(data_df.annual_inc, kde=True)
plt.title('Annual Income')
plt.xlabel("")
plt.ylabel("")
plt.show()


We observe that:
- The median annual income is USD 57K
- Income of most of the applicants lie below USD 80K

# Earliest Credit Line Year

The year the borrower's earliest reported credit line was opened


In [ ]:
data_df.earliest_cr_line_year.value_counts().plot()
plt.title('Earliest Credit Line Year')
plt.show()

From this description we can understand that the maximum borrowers have their credit line starting in the year 2000

# Earliest Credit Line Month 

The month the borrower's earliest reported credit line was opened


In [ ]:
data_df.earliest_cr_line_month.value_counts()

In [ ]:
data_df.earliest_cr_line_month.value_counts().plot()
plt.title('Earliest Credit Line Month')

From this graph we can understand that most of the people have prefer to open their credit line in the month of October

# Issued Year and month 

The month and the year in which loan was funded

In [ ]:
data_df.issue_d_year.value_counts()

In [ ]:
plt.hist(data_df.issue_d_year)
plt.title('Issued Year')
plt.show()

Most of the loans were issued in 2011

In [ ]:
data_df.issue_d_month.value_counts()

In [ ]:
data_df.issue_d_month.value_counts().plot()
plt.title('Issued Month')
plt.show()

Most of the loans were issued in the month of December to the borrowers

# Public Record Bankruptcies

Number of public record bankruptcies

In [ ]:
data_df.pub_rec_bankruptcies.value_counts()

From this values only we can understand that there are very few borrowers who have declared bankruptcy and fewer who have declared bankruptcy twice

# Last Credit Pulled date

The most recent month LC pulled the credit for this loan

In [ ]:
data_df.last_credit_pull_d.value_counts()


# Last Payment Amount

In [ ]:
data_df.last_pymnt_amnt.describe()

In [ ]:
plt.hist(data_df.last_pymnt_amnt)
plt.title("Last Payment Amount")
plt.show()

This graph  shows us that most of the borrower's last transaction was b/w 0-5000

# Last Payment date 

In [ ]:
data_df.last_pymnt_d.isna().sum()

# Collection Recovery Fee

In [ ]:
data_df.collection_recovery_fee.value_counts().plot()
plt.title('Collection Recovery fee')
plt.show()

From this chart we can understand that the collection recovery fee is 0 for most of the people and for the rest of them it's even less than 5000

# Recoveries

In [ ]:
data_df.recoveries.value_counts()

In [ ]:
data_df.recoveries.describe()

As we can this column has same observation most of the rows have 0 recovery and the rest have a minimum amunt

# Total Recovered Late Fee

In [ ]:
data_df.total_rec_late_fee.value_counts()


In [ ]:
data_df.total_rec_late_fee.describe()

Here also we can that the column doesn't have any significant data

# Total Recovery Interest

In [ ]:
data_df.total_rec_int.describe()

In [ ]:
data_df.total_rec_int.value_counts().plot()
plt.title('Total Recovery Interest')
plt.show()

# Total Recovery Principle 

In [ ]:
data_df.total_rec_prncp.describe()

In [ ]:
data_df['total_rec_prncp'].value_counts()

In [ ]:
plt.hist(data_df['total_rec_prncp'])
plt.title('Total recovery principle')
plt.show()
#data1['total_rec_late_fee'].value_counts()

# Total Payment Funded by Investors

Total payment received for the amount invested by the investors

In [ ]:
data_df.total_pymnt_inv.describe()

In [ ]:
#plt.boxplot(data_df.total_pymnt_inv)
plt.hist(data_df.total_pymnt_inv)
plt.title('Total Payment recieved by investors')
plt.show()

From this we can understand that the maximum payment the investors got for their investment was under 10000 USD

# Total Payment 

In [ ]:
data_df['total_pymnt'].value_counts()

In [ ]:
data_df.total_pymnt.describe()

In [ ]:
sns.boxplot(data_df['total_pymnt'])
plt.title('Total Payment Received')
plt.show()


In [ ]:
data_df.total_pymnt.median()

From this we can understand that the median amount of the total payment received is 9638 USD

# Out Principal Invested by Investors

Remaining outstanding principal for total amount funded by investors

In [ ]:
data_df.out_prncp_inv.describe()

In [ ]:
data_df.out_prncp.value_counts()

From the above data we can understand that most of the investors money has been paid

# Out Principal

Remaining outstanding principal for total amount funded


In [ ]:
data_df.out_prncp.describe()

In [ ]:
data_df.out_prncp.value_counts()

From these two insights we can understand that most of the people have 0 outstanding principle to be paid

# Total Accounts

The total number of credit lines currently in the borrower's credit file


In [ ]:
data_df.total_acc.value_counts()

In [ ]:
data_df.total_acc.describe()

In [ ]:
sns.boxplot(data_df.total_acc)
plt.title('Total Number of Credit Lines ')
plt.show()

# Revolving Credit Utilization 

Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.


In [ ]:
data1 = data_df.revol_util

In [ ]:
arr=[]
for i in data1:
        if type(i)== str:
            i= i.rstrip('%')
            arr.append(i)

In [ ]:
for i in range(len(arr)):
    arr[i] = float(arr[i])

In [ ]:
indi = data_df[data_df.revol_util.isna()].index.tolist()

In [ ]:
for j in range(len(indi)):
        arr.insert(indi[j],0)


In [ ]:
data_df.revol_util=arr

In [ ]:
data_df.revol_util.describe()

In [ ]:
plt.boxplot(data_df.revol_util)
plt.title('Revolving Credit utilization')
plt.show()

In [ ]:
sns.__version__

In [ ]:
sns.histplot(data_df.revol_util)

From this we can understand the percentage of the revolving credit utilization done by the borrower

# Revolving balance

Total credit revolving balance

In [ ]:
data_df.revol_bal.describe()

In [ ]:
plt.boxplot(data_df.revol_bal)
plt.title('Revolving Balance')
plt.show()

From the initial analysis we can understand that the median of the total revolving balance is less than 9000 USD

# Public Records

The number of derogatory public records

In [ ]:
data_df.pub_rec.describe()

In [ ]:
data_df.pub_rec.value_counts()



These number show that the maximum of the borrowers don't have derogaotry public records

# Open Credit lines

The number of open credit lines in the borrower's credit file.

In [ ]:
data_df.open_acc.describe()

In [ ]:
plt.boxplot(data_df.open_acc)
plt.title('Open Credit Lines')
plt.show()

A borrower has on an average 9 credit lines open on his/her name

## Step 2 Segmented Univariate Analysis

Amount of Revolving credit based on number of credit lines

Loan Amount based on purpose 

In [ ]:
data2 = data_df[['loan_status','purpose','loan_amnt']]

In [ ]:
data2.groupby(['loan_status','purpose']).sum()

In [ ]:
g = sns.scatterplot(x='loan_amnt',
               y='purpose',data=data2,
               hue='loan_status')
a = data2.purpose.value_counts().index


plt.title('Loan Amount based on the Purpose')

plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.show()

In [ ]:
a = data2.loan_amnt.sum()
fig = px.scatter(data2,x='purpose', y= 'loan_amnt', size = 'loan_amnt')
fig.show()

Relation of loan amount and Annual income

In [ ]:
loan_purpose= data_df.groupby('purpose').sum() 
loan_purpose

In [ ]:
fig=px.scatter(loan_purpose,x='annual_inc', y=loan_purpose.index, size='loan_amnt')
fig.show()